### Preprocessing

In [119]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.linear_model import LinearRegression

#  Import and read the charity_data.csv.
import pandas as pd 
vax_df = pd.read_csv("sample_data/county_data.csv")
income_df = pd.read_csv("sample_data/NYS_income")
vax_df = pd.read_csv("sample_data/county_group_vax.csv")
vax_df.head()
income_df.head()

,index,County / County Group,Households with Elderly,Households with Children,Economic Development Region,Income Groups,Percent of Poverty Level,Low-to-Moderate Income (LMI) Group,Race / Ethnicity,Housing Unit Type,LMI Population Segment,Education Level,Head of Household Age
0,0,Albany,No,No,Capital District,"$0 to <$10,000",1 - Income at or below 100% HHSPG,Group 1 - Very Low Income,"Asian, non-Hispanic",4 - Moderate Multi-Family (5-50 units),#1 – Low-Income Renters in Multifamily (5+ Uni...,3 - Some College,<30
1,1,Albany,Yes,No,Capital District,"$0 to <$10,000",1 - Income at or below 100% HHSPG,Group 1 - Very Low Income,"Asian, non-Hispanic",4 - Moderate Multi-Family (5-50 units),#1 – Low-Income Renters in Multifamily (5+ Uni...,4 - Associate's,70+
2,2,Albany,No,No,Capital District,"$0 to <$10,000",1 - Income at or below 100% HHSPG,Group 1 - Very Low Income,"Asian, non-Hispanic",5 - Large Multi-Family (50+ units),#1 – Low-Income Renters in Multifamily (5+ Uni...,4 - Associate's,<30
3,3,Albany,No,No,Capital District,"$0 to <$10,000",1 - Income at or below 100% HHSPG,Group 1 - Very Low Income,"Asian, non-Hispanic",4 - Moderate Multi-Family (5-50 units),#1 – Low-Income Renters in Multifamily (5+ Uni...,3 - Some College,<30
4,4,Albany,Yes,No,Capital District,"$0 to <$10,000",1 - Income at or below 100% HHSPG,Group 1 - Very Low Income,"Asian, non-Hispanic",2 - Single Family Detached,#3 – Low-Income Owners in Single-Family & Smal...,5 - Bachelor's,70+


In [120]:
#Only keep necessary columns
vax_df = vax_df[['county_group','population','metrics.vaccinationsCompletedRatio']]

#Drop rows with NA
vax_df = vax_df.dropna()

#Rename columns
vax_df.columns = ['County', 'Total_Pop', 'Vax_Rate']

vax_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 0 to 38
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   County     39 non-null     object 
 1   Total_Pop  39 non-null     int64  
 2   Vax_Rate   39 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ KB


In [121]:
# Drop non-beneficial columns
income_df = income_df.drop(['index','Economic Development Region', 'LMI Population Segment'], axis=1)

#Drop rows with NA
income_df = income_df.dropna()

#Recode categorical features
income_cat = ['Households with Elderly',
       'Households with Children', 'Income Groups', 'Percent of Poverty Level',
       'Low-to-Moderate Income (LMI) Group', 'Housing Unit Type',
       'Education Level', 'Head of Household Age', 'Race / Ethnicity']

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(income_df[income_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(income_cat)

# Merge one-hot encoded features and drop the originals
income_df = income_df.merge(encode_df,left_index=True, 
                                  right_index=True)
income_df = income_df.drop(income_cat,1)

#Rename County
income_df.rename(columns={'County / County Group':'County'}, inplace=True)

#Group by county
income_df = income_df.groupby('County').sum()
income_df.head()

,Households with Elderly_No,Households with Elderly_Yes,Households with Children_No,Households with Children_Yes,"Income Groups_$0 to <$10,000","Income Groups_$10,000-<$20,000","Income Groups_$20,000-<$30,000","Income Groups_$30,000-<$40,000","Income Groups_$40,000-<$50,000","Income Groups_$50,000+",Percent of Poverty Level_1 - Income at or below 100% HHSPG,Percent of Poverty Level_2 - Income 101%-150% HHSPG,Percent of Poverty Level_3 - Income 151%-200% HHSPG,Percent of Poverty Level_4 - Income 201% HHSPG or more,Low-to-Moderate Income (LMI) Group_Group 1 - Very Low Income,Low-to-Moderate Income (LMI) Group_Group 2 - Low Income,Low-to-Moderate Income (LMI) Group_Group 3 - Moderate Income,Low-to-Moderate Income (LMI) Group_Non-LMI Household,Housing Unit Type_1 - Single Family Attached,Housing Unit Type_2 - Single Family Detached,Housing Unit Type_3 - Small Multi-Family (2-4 units),Housing Unit Type_4 - Moderate Multi-Family (5-50 units),Housing Unit Type_5 - Large Multi-Family (50+ units),Housing Unit Type_6 - Mobile Homes & Other,Education Level_1 - Less than High School Diploma,Education Level_2 - High School Diploma,Education Level_3 - Some College,Education Level_4 - Associate's,Education Level_5 - Bachelor's,Education Level_6 - Graduate Degree,Head of Household Age_30-39,Head of Household Age_40-49,Head of Household Age_50-59,Head of Household Age_60-69,Head of Household Age_70+,Head of Household Age_<30,"Race / Ethnicity_Asian, non-Hispanic","Race / Ethnicity_Black, non-Hispanic",Race / Ethnicity_Hispanic,Race / Ethnicity_Other,"Race / Ethnicity_White, non-Hispanic"
County,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Albany,4142.0,2934.0,5458.0,1618.0,374.0,584.0,626.0,606.0,562.0,4324.0,640.0,436.0,456.0,5544.0,896.0,792.0,1152.0,4236.0,290.0,4124.0,1450.0,792.0,330.0,90.0,478.0,1574.0,1228.0,814.0,1554.0,1428.0,984.0,1164.0,1554.0,1380.0,1282.0,712.0,272.0,602.0,210.0,128.0,5864.0
Bronx,14442.0,8836.0,14960.0,8318.0,3606.0,3732.0,2634.0,2404.0,1840.0,9062.0,6372.0,2838.0,2338.0,11730.0,8208.0,3870.0,4004.0,7196.0,1442.0,1720.0,4336.0,7824.0,7884.0,72.0,5966.0,5826.0,4440.0,1754.0,3174.0,2118.0,3946.0,4716.0,5024.0,3858.0,3648.0,2086.0,922.0,6916.0,11292.0,404.0,3744.0
"Broome, Chenango, Delaware, & Tioga",5834.0,5616.0,8732.0,2718.0,786.0,1474.0,1342.0,1220.0,1098.0,5530.0,1446.0,1232.0,1126.0,7646.0,2200.0,1964.0,1538.0,5748.0,144.0,7852.0,1260.0,746.0,156.0,1292.0,1178.0,3692.0,2218.0,1480.0,1610.0,1272.0,1326.0,1662.0,2550.0,2530.0,2646.0,736.0,164.0,212.0,200.0,122.0,10752.0
Cattaraugus & Allegany,3790.0,3288.0,5156.0,1922.0,462.0,932.0,896.0,880.0,754.0,3154.0,922.0,856.0,802.0,4498.0,1406.0,1428.0,1008.0,3236.0,52.0,5324.0,586.0,258.0,28.0,830.0,706.0,2724.0,1352.0,870.0,768.0,658.0,890.0,1060.0,1582.0,1482.0,1518.0,546.0,34.0,28.0,34.0,192.0,6790.0
Cayuga & Onondaga,9460.0,7046.0,12010.0,4496.0,1146.0,1696.0,1630.0,1588.0,1390.0,9056.0,1894.0,1394.0,1436.0,11782.0,2706.0,2460.0,2126.0,9214.0,600.0,11442.0,1716.0,1600.0,596.0,552.0,1356.0,4172.0,3198.0,1990.0,3160.0,2630.0,2120.0,2710.0,3780.0,3312.0,3144.0,1440.0,344.0,1158.0,412.0,248.0,14344.0


In [122]:
# join files
df = vax_df.merge(income_df, how='inner',on='County')
df

,County,Total_Pop,Vax_Rate,Households with Elderly_No,Households with Elderly_Yes,Households with Children_No,Households with Children_Yes,"Income Groups_$0 to <$10,000","Income Groups_$10,000-<$20,000","Income Groups_$20,000-<$30,000","Income Groups_$30,000-<$40,000","Income Groups_$40,000-<$50,000","Income Groups_$50,000+",Percent of Poverty Level_1 - Income at or below 100% HHSPG,Percent of Poverty Level_2 - Income 101%-150% HHSPG,Percent of Poverty Level_3 - Income 151%-200% HHSPG,Percent of Poverty Level_4 - Income 201% HHSPG or more,Low-to-Moderate Income (LMI) Group_Group 1 - Very Low Income,Low-to-Moderate Income (LMI) Group_Group 2 - Low Income,Low-to-Moderate Income (LMI) Group_Group 3 - Moderate Income,Low-to-Moderate Income (LMI) Group_Non-LMI Household,Housing Unit Type_1 - Single Family Attached,Housing Unit Type_2 - Single Family Detached,Housing Unit Type_3 - Small Multi-Family (2-4 units),Housing Unit Type_4 - Moderate Multi-Family (5-50 units),Housing Unit Type_5 - Large Multi-Family (50+ units),Housing Unit Type_6 - Mobile Homes & Other,Education Level_1 - Less than High School Diploma,Education Level_2 - High School Diploma,Education Level_3 - Some College,Education Level_4 - Associate's,Education Level_5 - Bachelor's,Education Level_6 - Graduate Degree,Head of Household Age_30-39,Head of Household Age_40-49,Head of Household Age_50-59,Head of Household Age_60-69,Head of Household Age_70+,Head of Household Age_<30,"Race / Ethnicity_Asian, non-Hispanic","Race / Ethnicity_Black, non-Hispanic",Race / Ethnicity_Hispanic,Race / Ethnicity_Other,"Race / Ethnicity_White, non-Hispanic"
0,Albany,305506,0.67200,4142.0,2934.0,5458.0,1618.0,374.0,584.0,626.0,606.0,562.0,4324.0,640.0,436.0,456.0,5544.0,896.0,792.0,1152.0,4236.0,290.0,4124.0,1450.0,792.0,330.0,90.0,478.0,1574.0,1228.0,814.0,1554.0,1428.0,984.0,1164.0,1554.0,1380.0,1282.0,712.0,272.0,602.0,210.0,128.0,5864.0
1,Bronx,1418207,0.61400,14442.0,8836.0,14960.0,8318.0,3606.0,3732.0,2634.0,2404.0,1840.0,9062.0,6372.0,2838.0,2338.0,11730.0,8208.0,3870.0,4004.0,7196.0,1442.0,1720.0,4336.0,7824.0,7884.0,72.0,5966.0,5826.0,4440.0,1754.0,3174.0,2118.0,3946.0,4716.0,5024.0,3858.0,3648.0,2086.0,922.0,6916.0,11292.0,404.0,3744.0
2,"Broome, Chenango, Delaware, & Tioga",330033,0.55050,5834.0,5616.0,8732.0,2718.0,786.0,1474.0,1342.0,1220.0,1098.0,5530.0,1446.0,1232.0,1126.0,7646.0,2200.0,1964.0,1538.0,5748.0,144.0,7852.0,1260.0,746.0,156.0,1292.0,1178.0,3692.0,2218.0,1480.0,1610.0,1272.0,1326.0,1662.0,2550.0,2530.0,2646.0,736.0,164.0,212.0,200.0,122.0,10752.0
3,Cattaraugus & Allegany,122208,0.44450,3790.0,3288.0,5156.0,1922.0,462.0,932.0,896.0,880.0,754.0,3154.0,922.0,856.0,802.0,4498.0,1406.0,1428.0,1008.0,3236.0,52.0,5324.0,586.0,258.0,28.0,830.0,706.0,2724.0,1352.0,870.0,768.0,658.0,890.0,1060.0,1582.0,1482.0,1518.0,546.0,34.0,28.0,34.0,192.0,6790.0
4,Cayuga & Onondaga,537104,0.60200,9460.0,7046.0,12010.0,4496.0,1146.0,1696.0,1630.0,1588.0,1390.0,9056.0,1894.0,1394.0,1436.0,11782.0,2706.0,2460.0,2126.0,9214.0,600.0,11442.0,1716.0,1600.0,596.0,552.0,1356.0,4172.0,3198.0,1990.0,3160.0,2630.0,2120.0,2710.0,3780.0,3312.0,3144.0,1440.0,344.0,1158.0,412.0,248.0,14344.0
5,Chautauqua,126903,0.53000,2654.0,2348.0,3806.0,1196.0,332.0,682.0,610.0,598.0,542.0,2238.0,640.0,618.0,560.0,3184.0,980.0,1000.0,672.0,2350.0,68.0,3754.0,570.0,202.0,40.0,368.0,504.0,1740.0,980.0,694.0,604.0,480.0,600.0,710.0,1200.0,1054.0,1130.0,308.0,22.0,50.0,156.0,42.0,4732.0
6,"Clinton, Franklin, Essex & Hamilton",171808,0.66975,3158.0,3042.0,4696.0,1504.0,432.0,704.0,674.0,622.0,656.0,3112.0,762.0,560.0,558.0,4320.0,1092.0,990.0,866.0,3252.0,104.0,4504.0,444.0,310.0,68.0,770.0,620.0,2098.0,1228.0,666.0,826.0,762.0,686.0,948.0,1416.0,1418.0,1366.0,366.0,28.0,30.0,44.0,112.0,5986.0
7,Columbia & Greene,106649,0.62300,1702.0,1900.0,2784.0,818.0,184.0,364.0,338.0,356.0,290.0,2070.0,352.0,286.0,324.0,2640.0,520.0,542.0,420.0,2120.0,58.0,2764.0,314.0,150.0,48.0,268.0,318.0,972.0,720.0,418.0,572.0,602.0,350.0,558

In [123]:
# Drop county
df = df.drop(['County'], axis=1)

In [124]:
# Check all values are numeric
df.info()
df.columns


<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 0 to 38
Data columns (total 43 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   Total_Pop                                                     39 non-null     int64  
 1   Vax_Rate                                                      39 non-null     float64
 2   Households with Elderly_No                                    39 non-null     float64
 3   Households with Elderly_Yes                                   39 non-null     float64
 4   Households with Children_No                                   39 non-null     float64
 5   Households with Children_Yes                                  39 non-null     float64
 6   Income Groups_$0 to <$10,000                                  39 non-null     float64
 7   Income Groups_$10,000-<$20,000                                39 non-null

Index(['Total_Pop', 'Vax_Rate', 'Households with Elderly_No',
       'Households with Elderly_Yes', 'Households with Children_No',
       'Households with Children_Yes', 'Income Groups_$0 to <$10,000',
       'Income Groups_$10,000-<$20,000', 'Income Groups_$20,000-<$30,000',
       'Income Groups_$30,000-<$40,000', 'Income Groups_$40,000-<$50,000',
       'Income Groups_$50,000+',
       'Percent of Poverty Level_1 - Income at or below 100% HHSPG',
       'Percent of Poverty Level_2 - Income 101%-150% HHSPG',
       'Percent of Poverty Level_3 - Income 151%-200% HHSPG',
       'Percent of Poverty Level_4 - Income 201% HHSPG or more',
       'Low-to-Moderate Income (LMI) Group_Group 1 - Very Low Income',
       'Low-to-Moderate Income (LMI) Group_Group 2 - Low Income',
       'Low-to-Moderate Income (LMI) Group_Group 3 - Moderate Income',
       'Low-to-Moderate Income (LMI) Group_Non-LMI Household',
       'Housing Unit Type_1 - Single Family Attached',
       'Housing Unit Type_2 - S

In [125]:
# Export to CSV for multiple regression
df.to_csv('sample_data/clean_df.csv')

In [126]:
# Split our preprocessed data into our features and target arrays
y = df["Vax_Rate"].values
X = df.drop(["Vax_Rate"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [127]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Neural Network

In [128]:
# Define the model - neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                          input_dim=number_input_features, 
                          activation="tanh")
)

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="linear"))

# Check the structure of the model
nn.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_41 (Dense)             (None, 10)                430       
_________________________________________________________________
dense_42 (Dense)             (None, 1)                 11        
Total params: 441
Trainable params: 441
Non-trainable params: 0
_________________________________________________________________


In [129]:
# Compile the model
nn.compile(loss="mean_absolute_error", 
           optimizer="adam")

In [130]:
# Create a callback that saves the model's weights every 5 epochs
callbacks = tf.keras.callbacks.ModelCheckpoint('weights{epoch:08d}.h5', 
                                     save_weights_only=True, save_freq=5)

In [131]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[callbacks])

Epoch 1/100
1/1 [==============================] - 0s 308ms/step - loss: 0.8457
Epoch 2/100
1/1 [==============================] - 0s 9ms/step - loss: 0.7881
Epoch 3/100
1/1 [==============================] - 0s 4ms/step - loss: 0.7304
Epoch 4/100
1/1 [==============================] - 0s 5ms/step - loss: 0.6724
Epoch 5/100
1/1 [==============================] - 0s 29ms/step - loss: 0.6169
Epoch 6/100
1/1 [==============================] - 0s 7ms/step - loss: 0.5652
Epoch 7/100
1/1 [==============================] - 0s 5ms/step - loss: 0.5260
Epoch 8/100
1/1 [==============================] - 0s 6ms/step - loss: 0.4943
Epoch 9/100
1/1 [==============================] - 0s 6ms/step - loss: 0.4775
Epoch 10/100
1/1 [==============================] - 0s 12ms/step - loss: 0.4690
Epoch 11/100
1/1 [==============================] - 0s 6ms/step - loss: 0.4778
Epoch 12/100
1/1 [==============================] - 0s 6ms/step - loss: 0.4869
Epoch 13/100
1/1 [==============================] - 0s 4m

In [132]:
# Evaluate the model using the test data
model_loss = nn.evaluate(X_test_scaled,y_test,verbose=1)
print(f"Loss: {model_loss}")

1/1 [==============================] - 0s 84ms/step - loss: 0.1382
Loss: 0.138167142868042


In [133]:
#Save 
nn.save('COVID_Vax_Predictions.h5')